<a href="https://colab.research.google.com/github/scalgary92/HelpMe/blob/master/ar106.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**<font color='blue'>You can mount your Google Drive</font>**

In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [3]:
import os
import pandas as pd
import numpy as np
from google.cloud import bigquery
#we need a google project - if you have no ipsos right - set up a project on your personal google account
#so you can try this
my_project_id = 'mgcp-1192365-ipsos-gbht-srf617'

client = bigquery.Client(project=my_project_id)

from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [4]:
def extract_data(columns,whereclause):
    # Initialisez un client BigQuery

    query = f"""
    BEGIN
    DECLARE strSQLFlat STRING;
    DECLARE strSQLTall STRING;
    DECLARE strSQLTabulate STRING;
    DECLARE strSQLFullCommand STRING;
    DECLARE strSQLWhereClause STRING;

    SET strSQLTall = CONCAT("WITH BPtall AS (",
        "SELECT * FROM (",CHR(13),CHR(10),
            " SELECT m.metricLabel, r.responseLabel, NULL AS intData, NULL AS floatData, NULL AS DateData, NULL AS StringData, c.respondentUUID, m.metricCode, r.responseCode, m.metricOrder, r.responseOrder, m.metricID, r.responseID, m.metricType",CHR(13),CHR(10),
            " FROM BP_COMMON.tblMetricMap m",CHR(13),CHR(10),
            " INNER JOIN BP_COMMON.tblResponseMap r ON m.metricID=r.metricID",CHR(13),CHR(10),
            " INNER JOIN BP_COMMON.rldCategoricals c ON c.metricID=m.metricID AND c.responseID=r.responseID",CHR(13),CHR(10),
            " WHERE m.metricCode IN  UNNEST({columns})",CHR(13),CHR(10),
            " UNION ALL",CHR(13),CHR(10),
            " SELECT m.metricLabel, NULL AS responseLabel, o.intData, o.floatData, o.DateData, o.StringData, o.respondentUUID, m.metricCode, NULL AS responseCode, m.metricOrder, NULL AS responseOrder, m.metricID, NULL AS responseID, m.metricType",CHR(13),CHR(10),
            " FROM BP_COMMON.tblMetricMap m",CHR(13),CHR(10),
            " INNER JOIN BP_COMMON.rldOther o ON o.metricID=m.metricID",CHR(13),CHR(10),
            " WHERE m.metricCode IN UNNEST({columns})",CHR(13),CHR(10),
            " ORDER BY metricOrder, responseOrder)),");
    SET strSQLFlat=(
    WITH fieldList AS (
    SELECT * FROM UNNEST ({columns}) AS strVarName
    )
    SELECT STRING_AGG(strSQL ORDER BY ord)
    FROM (
    SELECT CONCAT(" bpFlat AS (","SELECT respondentUUID ",CHR(13),CHR(10)) AS strSQL,-1 AS ord
    UNION ALL
    SELECT CONCAT("MAX(IF(metricCode = '",metricCode,"',",CASE WHEN metricType="long" THEN "intData" WHEN metricType="text" THEN "stringData" WHEN metricType="double" THEN "floatData" WHEN metricType="date" THEN "dateData" END ,", NULL)) as ",metricCode,CHR(13),CHR(10)) AS strSQL, metricOrder AS ord
    FROM BP_COMMON.tblMetricMap m INNER JOIN fieldList ON fieldList.strVarname=m.metricCode AND metricType<>"categorical"
    UNION ALL
    SELECT CONCAT("MAX(IF(metricCode = '",metricCode,"',",CASE WHEN metricType="categorical" THEN "responseLabel" END ,", NULL)) as ",metricCode,CHR(13),CHR(10)) AS strSQL, metricOrder AS ord
    FROM BP_COMMON.tblMetricMap m INNER JOIN fieldList ON fieldList.strVarname=m.metricCode AND metricType="categorical"
    LEFT OUTER JOIN (SELECT metricCodeSegment, COUNT(DISTINCT metricCode) AS mpCount FROM BP_COMMON.tblMetricMap WHERE isMultipunch<>0 GROUP BY 1 HAVING COUNT(DISTINCT metricCode)=1) mpNotSpread ON m.metricCodeSegment=mpNotSpread.metricCodeSegment
    WHERE mpNotSpread.mpCount IS NULL /* If these are multipunch, they were spread out in the metricMap */
    UNION ALL
    SELECT CONCAT("STRING_AGG(DISTINCT IF(metricCode = '",metricCode,"',",CASE WHEN metricType="categorical" THEN "responseLabel" END ,", NULL),'|') as ",metricCode,CHR(13),CHR(10)) AS strSQL, metricOrder AS ord
    FROM BP_COMMON.tblMetricMap m INNER JOIN fieldList ON fieldList.strVarname=m.metricCode AND metricType="categorical"
    LEFT OUTER JOIN (SELECT metricCodeSegment, COUNT(DISTINCT metricCode) AS mpCount FROM BP_COMMON.tblMetricMap WHERE isMultipunch<>0 GROUP BY 1 HAVING COUNT(DISTINCT metricCode)=1) mpNotSpread ON m.metricCodeSegment=mpNotSpread.metricCodeSegment
    WHERE mpNotSpread.mpCount IS NOT NULL /* These multipunch only appear one time in the metricMap, so would have multiple answers */
    UNION ALL
    SELECT CONCAT(" FROM bpTall",CHR(13),CHR(10),"GROUP BY respondentUUID ORDER BY respondentUUID)") AS strSQL, 999999999 AS ord
    ));

    SET strSQLWhereClause="{whereclause}";

    SET strSQLTabulate =
    CONCAT(CHR(13),CHR(10),CHR(13),CHR(10),"SELECT * ",CHR(13),CHR(10),
    " FROM BPFlat ",CHR(13),CHR(10),
    " ",strSQLWhereClause,CHR(13),CHR(10));



    /* Tabulate the results, or show the SQL */
    SET strSQLFullCommand = CONCAT(strSQLTall,strSQLFlat,strSQLTabulate);
    EXECUTE IMMEDIATE strSQLFullCommand;




 END
"""

    # Utilisez le client BigQuery pour exécuter la requête et convertir le résultat en un DataFrame pandas
    df = client.query(query).to_dataframe()

    # Retournez le DataFrame
    return df



In [ ]:

variables = ['respondentUUID',
             "bd_matchKey",
'hv_sampleType',
'wt_main',
'fv_surveyWave',
'bd_country',
'bd_ageNets1',

'qq_m_familiarity_google',
'qq_m_advocacy_google',
'qq_m_trust_google',

'qq_m_contribAttr_google_benefitsSocietyCulture',
'qq_m_contribAttr_google_committed',
'qq_m_contribAttr_google_crisesAid',
'qq_m_contribAttr_google_devAiTech',
'qq_m_contribAttr_google_economy',
'qq_m_contribAttr_google_education',
'qq_m_contribAttr_google_environmentHealth',
'qq_m_contribAttr_google_equalRights',
'qq_m_contribAttr_google_freedomOfExpression',
'qq_m_contribAttr_google_helpFindJobs',
'qq_m_contribAttr_google_helpfulEdu',
'qq_m_contribAttr_google_helpsSmallBusGrow',
'qq_m_contribAttr_google_improveCompSciEdu',
'qq_m_contribAttr_google_improveSkills',
'qq_m_contribAttr_google_improvesPeoplesLives',
'qq_m_contribAttr_google_jobsGrowth',
'qq_m_contribAttr_google_partOfCulture',
'qq_m_contribAttr_google_positiveContrSociety',
'qq_m_contribAttr_google_provideCompSciEdu',
'qq_m_contribAttr_google_responsibleWithAi',
'qq_m_contribAttr_google_saveTime',
'qq_m_contribAttr_google_sustainableChoices',
'qq_m_contribAttr_google_userFriendly',

'qq_m_privacyAttr_google_cybersecurity',
'qq_m_privacyAttr_google_misinformation',
'qq_m_privacyAttr_google_noPiiSales',
'qq_m_privacyAttr_google_parentsTools',
'qq_m_privacyAttr_google_peopleSafeOnline',
'qq_m_privacyAttr_google_piiSafe',
'qq_m_privacyAttr_google_piiToolsMngmnt',
'qq_m_privacyAttr_google_preventViolentContent',
'qq_m_privacyAttr_google_respectsPrivacy',
'qq_m_privacyAttr_google_safeData',
'qq_m_privacyAttr_google_safeForKids',
'qq_m_privacyAttr_google_securityTech',
'qq_m_privacyAttr_google_transparent',

'qq_m_funcEmotAttr_google_accurate',
'qq_m_funcEmotAttr_google_aiLeader',
'qq_m_funcEmotAttr_google_aiTech',
'qq_m_funcEmotAttr_google_cares',
'qq_m_funcEmotAttr_google_caring',
'qq_m_funcEmotAttr_google_clw',
'qq_m_funcEmotAttr_google_cuttingEdge',
'qq_m_funcEmotAttr_google_enjoyLife',
'qq_m_funcEmotAttr_google_enviSustain',
'qq_m_funcEmotAttr_google_ethical',
'qq_m_funcEmotAttr_google_forEveryone',
'qq_m_funcEmotAttr_google_forMe',
'qq_m_funcEmotAttr_google_fun',
'qq_m_funcEmotAttr_google_helpful',
'qq_m_funcEmotAttr_google_helpfulProds',
'qq_m_funcEmotAttr_google_helpsImproveLives',
'qq_m_funcEmotAttr_google_innoMotiv',
'qq_m_funcEmotAttr_google_innovative',
'qq_m_funcEmotAttr_google_inspiring',
'qq_m_funcEmotAttr_google_open',
'qq_m_funcEmotAttr_google_optimistic',
'qq_m_funcEmotAttr_google_peopleSafe',
'qq_m_funcEmotAttr_google_privacy',
'qq_m_funcEmotAttr_google_quality',
'qq_m_funcEmotAttr_google_responsible',
'qq_m_funcEmotAttr_google_safe',
'qq_m_funcEmotAttr_google_saveTime',
'qq_m_funcEmotAttr_google_trustworthy',
'qq_m_funcEmotAttr_google_userFriendly',
'qq_m_funcEmotAttr_google_values',
'qq_m_funcEmotAttr_google_visionary'


]

clause= "WHERE hv_sampleType='MAIN' \
AND EXTRACT(YEAR FROM DATE(fv_surveyWave)) =2023 \
AND EXTRACT(MONTH FROM DATE(fv_surveyWave)) in (4) \
AND bd_country in('India') ORDER BY fv_surveyWave ,bd_country, bd_matchKeyDESC "

In [ ]:
df = extract_data(variables,clause)
df.to_csv('testS_3.csv')